# Guardian - Iteration 10

## Import Packages

In [1]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [2]:
iteration = 10

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2018-04-01",
    "to-date": "2018-09-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY2"),
}


In [3]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [4]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [5]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2018-04-01&to-date=2018-09-01&order-by=newest&api-key=934fc87f-0792-4733-b5ed-59d5aa136db0&page=1
status ok
total 33744
startIndex 1
pageSize 50
pages 675
orderBy newest
---- RUNTIME STATUS ---- 
(1/675) in 0:00:28.947560s
(2/675) in 0:00:13.622299s
(3/675) in 0:00:17.207671s
(4/675) in 0:00:10.616437s
(5/675) in 0:00:04.928813s
(6/675) in 0:00:06.005889s
(7/675) in 0:00:18.189178s
(8/675) in 0:00:30.204453s
(9/675) in 0:00:18.075731s
(10/675) in 0:00:19.991696s
(11/675) in 0:00:11.370920s
(12/675) in 0:00:24.445267s
(13/675) in 0:00:06.659883s
(14/675) in 0:00:14.036164s
(15/675) in 0:00:04.792988s
(16/675) in 0:00:09.443239s
(17/675) in 0:00:10.847310s
(18/675) in 0:00:09.797471s
(19/675) in 0:00:07.078449s
(20/675) in 0:00:05.258442s
(21/675) in 0:00:13.559429s
(22/675) in 0:00:09.630636s
(23/675) in 0:00:05.187293s
(24/675) in 0:00:11.389691s
(25/675) i

In [6]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}_1.csv.zip", compression="zip")
    return df

In [7]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(33744, 13) took 0:00:00.010145s

unfold_columns:
 shape=(33744, 50) took 0:00:07.550243s

drop_columns:
 shape=(33744, 32) took 0:00:00.026938s

create_csv:
 shape=(33744, 32) took 0:00:23.943667s

